**Importing some useful Libraries & Data**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

df = pd.read_csv('../input/devanagiri-characters/data.csv')
df.head()

In [3]:
# Printing each character in data with its title:

char = df['character'].value_counts().index
char = char.tolist()

for name in char:
    plt.figure(figsize=(4,4))
    image = (df[df['character']==name].head(1))
    image = image.iloc[:,:-1].values.reshape(32,32).astype('int32')         # Size of img - 32x32
    plt.imshow(image)
    plt.title(name)

**Now we make a matrix**

In [4]:
X = df.iloc[:,:-1]
y = df['character']

**Transforming the data to feed to the model.**

In [5]:
# Shape: (data.shape, size of image, feature map)
X = X.values.reshape(X.shape[0],32,32,1)

# LabelBinarizer is like one-hot-encoding 
# OneHotEncoder needs data in integer encoded form first to convert into its respective encoding which is not required in the case of LabelBinarizer.
y = LabelBinarizer().fit_transform(y)

**Splitting data into train and test set.**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,shuffle=True,random_state=101)

X_train, X_test = X_train/255, X_test/255

**Convolutional Layers**

In [7]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), input_shape=(32,32,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(.015))

model.add(Flatten())
model.add(Dense(46, activation='softmax'))
model.summary()

**Compiling the Model**

In [8]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

**Training the Model**

In [9]:
model.fit(X_train,y_train, epochs=15)

**Accuracy - 98%**